In [18]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, AveragePooling2D, Conv2D, Flatten
from keras.layers import Lambda
from keras import backend as K # tensorflow

import coremltools
from coremltools.proto import NeuralNetwork_pb2

import os

In [19]:
kears_file = "./KerasMNIST_customlayer.h5"
coreml_file = './KerasMNIST_customlayer.mlmodel'

In [20]:
# Just sigmoid
def custom_sigmoid_activation(x):
    return K.sigmoid(x)

# Just relu
def custom_relu_activation(x):
    return K.relu(x)

In [21]:
def build_and_learn_keras_model():


    batch_size = 128
    num_classes = 10
    epochs = 1

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    print(x_train.shape)
    print(x_train.shape[1:])

    img_rows = 28
    img_cols = 28

    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # 入力値の正規化
    x_train /= 255
    x_test /= 255

    # 教師データをクラス分類のデータに変換
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # ネットワーク設計
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', input_shape=input_shape))

    # custom layer
    model.add(Lambda(custom_sigmoid_activation))

    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(128))

    # custom layer
    model.add(Lambda(custom_relu_activation))

    model.add(Dense(num_classes, activation='softmax'))

    # ネットワークの構成を出力する
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [22]:
if not os.path.exists(kears_file):
    keras_model = build_and_learn_keras_model()
    keras_model.save(kears_file)

In [23]:
def convert_custom_lambda_layer(layer):
    
    params = NeuralNetwork_pb2.CustomLayerParams()
    
    if layer.function.__name__ == custom_relu_activation.__name__:
        params.className = "custom_relu_activation"
        params.description = "RELU"
        return params
    elif layer.function.__name__ == custom_sigmoid_activation.__name__:
        params.className = "custom_sigmoid_activation"
        params.description = "sigmoid"
        return params
    else:
        return None

In [24]:
coreml_model = coremltools.converters.keras.convert(
    kears_file,
    input_names='image',
    output_names='digit',
    add_custom_layers=True,
    custom_conversion_functions={ "Lambda": convert_custom_lambda_layer }
)

coreml_model.author = u'Yuichi Yoshida'
coreml_model.license = 'MIT'
coreml_model.short_description = u'Custom layerのサンプル'

coreml_model.input_description['image'] = u'入力画像'
coreml_model.output_description['digit'] = u'推定した数字の確率'

coreml_model.save(coreml_file)

0 : conv2d_4_input, <keras.engine.topology.InputLayer object at 0x13f333b10>
1 : conv2d_4, <keras.layers.convolutional.Conv2D object at 0x13f030f50>
2 : lambda_3, <keras.layers.core.Lambda object at 0x13fa2a3d0>
3 : conv2d_5, <keras.layers.convolutional.Conv2D object at 0x13f0683d0>
4 : conv2d_5__activation__, <keras.layers.core.Activation object at 0x13fe21fd0>
5 : max_pooling2d_2, <keras.layers.pooling.MaxPooling2D object at 0x13fa3ae50>
6 : conv2d_6, <keras.layers.convolutional.Conv2D object at 0x13cc97690>
7 : conv2d_6__activation__, <keras.layers.core.Activation object at 0x13fd49ad0>
8 : flatten_2, <keras.layers.core.Flatten object at 0x13cd03f50>
9 : dense_3, <keras.layers.core.Dense object at 0x13cd03c50>
10 : lambda_4, <keras.layers.core.Lambda object at 0x13fa1de10>
11 : dense_4, <keras.layers.core.Dense object at 0x13cd22590>
12 : dense_4__activation__, <keras.layers.core.Activation object at 0x14035bd10>
